In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [2]:
np.random.seed(0)
torch.manual_seed(0)
# load data and make training set
data = torch.load('traindata.pt')
input = torch.from_numpy(data[3:, :-1])
print(input.shape)
print(data.shape)
i=0
print(input[i*10: (i+1)*10, :].shape)

torch.Size([97, 999])
(100, 1000)
torch.Size([10, 999])


In [9]:
np.random.seed(0)
torch.manual_seed(0)
# load data and make training set
data = torch.load('traindata.pt')
input = torch.from_numpy(data[3:, :-1])
target = torch.from_numpy(data[3:, 1:])
test_input = torch.from_numpy(data[:3, :-1])
test_target = torch.from_numpy(data[:3, 1:])
# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8, max_iter=5)
#begin to train


# flag to reset the counter
flag        = 0
step_counter = 0

while step_counter  < 15:
    #print('STEP: ', step_counter)
    
    def closure():
        global step_counter
        
        optimizer.zero_grad()
        out     = seq(input)
        start_n = (step_counter%15)*10
        end_n   = start_n+10
        out     = seq(input[start_n:end_n, :])
        target_mini = target[start_n:end_n, :]
        loss = criterion(out, target_mini)
        #print('loss:', loss.item())
        loss.backward()
        
        return loss
    
    loss = optimizer.step(closure)
    
#     if step_counter == 0 and flag == 0:
#         step_counter    = -1
#         flag = 1
        
    step_counter = step_counter+1
        
    print('loss: ', float(loss))
    
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        #print('test loss:', loss.item())
        y = pred.detach().numpy()
    # draw the result
    plt.figure(figsize=(30,10))
    plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    def draw(yi, color):
        plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
        plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b')
    plt.savefig('predict%d.pdf'%i)
    plt.close()

loss:  0.5025159704433867
loss:  0.48794905125337323
loss:  0.4850238664121137
loss:  0.4610036254105826
loss:  0.44233637044147156
loss:  0.41684108890697646
loss:  0.37979144441106655
loss:  0.3271568117194526
loss:  0.2581512367759698
loss:  0.17424084624837718


RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)